In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import os

In [53]:
#track_df = pd.read_csv()
temp_path = os.getcwd()
root_path = temp_path.split('/napster_2')[0]
repo_path = '/napster_2/lyric_genius_api/practice_data.csv'
practice_data_path = root_path + repo_path




In [54]:
track_df = pd.read_csv(practice_data_path)
print(track_df.genre.unique())
rel_columns = ['track_id', 'track_name_x', 'artist_name_x']
track_df = track_df[rel_columns]
track_df.rename(columns={'track_id': 'spotify_id',
                        'track_name_x': 'track_name',
                        'artist_name_x': 'artist_name'
})
#track_df.sample(30, random_state=15)


['alt-rock' 'latin' 'country' 'classical' 'hip-hop' 'edm' 'heavy-metal']


,spotify_id,track_name,artist_name
0,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood
1,2K7xn816oNHJZ0aVqdQsha,Softcore,The Neighbourhood
2,5E30LdtzQTGqRvNd7l6kG5,Daddy Issues,The Neighbourhood
3,2iUmqdfGZcHIhS3b9E9EWq,Everybody Talks,Neon Trees
4,7zwn1eykZtZ5LODrf7c0tS,You Get Me So High,The Neighbourhood
...,...,...,...
7045,1XgpK29CGGjZnxPYkiRbh4,Hey DJ,CNCO
7046,5q2MjTDby29ZOEigCVV28a,In My Hood,South Park Mexican
7047,0XpMTExp5q4nLZZZ3msDGn,La Funka,Ozuna
7048,0ulsRBiciReng91DhfVT9D,Bebé,Ozuna


In [55]:
# steps for Rocchio Feedback Filter
# PROCESS 1 convert the raw lyrics into the concept space.
# 1. Create a TFIDF vectorizer
# 2. Create a document term matrix using TFIDF vec fit_transform using the raw lyrics. [I think there is an option to lemmatize here]
# 3. Complete latent semantic indexing using TruncatedSVD(num components = num comncepts, specifiy the random state)
# 4. Fit the document term matrix using TruncatedSVD.fit_transform. THESE ARE YOUR VECTORS FOR SIMILARITY SCORING

# PROCESS 2 convert the query into a vector
# 1. Convert querry into a raw string
# 2. Use the TFIDF vectorizer above to transform the querry
# 3. Use the LSI object above to convert the querry into the concept space.

# PROCESS 3 execute the search
# 1. Find the cosine similarity between the querry and all lyrics
# 2. Sort the tracks by similarity
# 3. Return the top N tracks to the user.

# PROCESS 4 Rochhio Feedback Filtering
# 1. Group user feeback by love, no answer, dislike
# 2. Calculate the mean for each group
# 3. Apply alpha, beta, gamma, and phi to:
#       Original search, loves, hates, nuetral
# 4. Update the lyric search querry vector and return new results!



In [56]:
# Tuning Parameters
# minimum document frequency
min_df = 10
num_concepts = 100
# step 1 import old lyrical data into a dataframe.
lyric_df = pd.read_csv(practice_data_path)
lyric_df = lyric_df[lyric_df['genre'] != 'latin']
lyric_df = lyric_df[['track_name_x', 'artist_name_x', 'track_id','lyric_raw']]
lyric_df.rename(columns={
    'track_name_x': 'track_name',
    'artist_name_x': 'artist_name'
}, inplace=True)
lyric_df = lyric_df.dropna()
# replace new line character
lyric_df['lyric_raw'].replace('\n', ' ',regex=True, inplace=True)
# remove embed text from lyric genius API
lyric_df['lyric_raw'].replace('[0-9]{1,3}Embed', '', regex=True, inplace=True)
# create vectorizer object
vecObj = TfidfVectorizer(tokenizer=str.split, min_df=min_df)
# fit the TFIDF vectorizer
docTermMat = vecObj.fit_transform(lyric_df['lyric_raw'])



In [57]:
lsiObj = TruncatedSVD(n_components=num_concepts, random_state=42)
docVecs = lsiObj.fit_transform(docTermMat)

In [58]:
# part 2 convert the query into a vector in the concept space
user_query = 'Jealousy, turning saints into the sea Swimming through sick lullabies Choking on your alibis'
# vectorize
userVec = vecObj.transform([user_query])
# convert query vec into the concept space
userLsi = lsiObj.transform(userVec)


In [59]:
# part 3 execute search using cosine similarity
# 1. Find the cosine similarity between the query and all lyrics
# 2. Sort the tracks by similarity
# 3. Return the top N tracks to the user.

# calculate cosine similarity between every track and the lyric provided.
simVals = cosine_similarity(docVecs, userLsi)
# create a track name, track id, artist name, similarity dataframe
lyric_df['similarity'] = simVals

# this step is important, 
# the lyrics df is officially out of sync now, 
# the indexes need to be sorted again OR two copies need to be maintained
sim_df = lyric_df.sort_values(by='similarity', ascending=False)



In [60]:
sim_df.head(30)[['track_name', 'artist_name', 'track_id']]

,track_name,artist_name,track_id
4077,Eden,Battle Beast,68sLf8iqi43JZVw3eOdzwy
4386,Raise Your Fists,Battle Beast,2tEeM4xMbMpQnhQ2rbB2Rv
4103,Beyond the Burning Skies,Battle Beast,3Q0mfEn7EtB6c2Wyzz73FJ
4239,Into the Heart of Danger,Battle Beast,67kZ9zlALZr9rQKujKWw35
4197,Jade,X JAPAN,5BtD86hovw9uud7JkQID1H
3880,Clap Your Hands,Kungs,7g7OshelYRRnOSEMmisJVI
2879,Drunk on Your Love,Brett Eldredge,1jNNHFZmRGXZFHlil5uhei
4825,The Banshee,Enforcer,443tWeHzgruSSaq8DuDyE3
295,The High Road,Broken Bells,5cwN1htZyoWqZCc33f3RfE
4332,Katana,Enforcer,568IRlyLDeDufcY8Kl13p2
